# Gradient Descent Practice

In this repository, you are expecting to see the following analyses from scratch:

* __Model 1__: use a stat package in python & regularization <br/>
* __Model 2__: write gradient descent from scratch  <br/>
* __Model 3__: add regularization in model 2 <br/>
* __Model 4__: stochastic gradient descent (later)  <br/>
* __Model 5__: mini-batch gradient descent (later)  <br/>

### 1. Select the dataset: iris dataset

In [1]:
# import packages
import random
import math
import pandas as pd
import numpy as np
from sklearn import datasets

In [2]:
# read in the sample dataset from sklearn
iris = datasets.load_iris()
dat = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
# for simplicity, we only examine binary classification:
# map to whether not 1
value_map = {0. : 1, 
             1. : 0, 
             2. : 1} 
# replace the value in target & change column names
dat['target'] = dat['target'].map(value_map)
dat.columns = [var.replace(' (cm)', '') for var in dat.columns if '(cm)' in var] + ['target']
dat.head(3)

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1


In [3]:
dat.drop(['petal length', 'petal width'], axis = 1, inplace = True)

In [4]:
dat.columns

Index(['sepal length', 'sepal width', 'target'], dtype='object')

### Model 1: Use statistical packages

In [5]:
# Package 1: statsmodels.api
import statsmodels.api as sm
dat1 = dat.copy()
dat1['intercept'] = 1 # Note, if we don't have the intercept, statsmodels will by default has not intercept
model = sm.Logit(dat1['target'], dat1.loc[:,dat1.columns!='target'])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.506818
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                  150
Model:                          Logit   Df Residuals:                      147
Method:                           MLE   Df Model:                            2
Date:                Sun, 15 Jul 2018   Pseudo R-squ.:                  0.2038
Time:                        16:56:06   Log-Likelihood:                -76.023
converged:                       True   LL-Null:                       -95.477
                                        LLR p-value:                 3.557e-09
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
sepal length    -0.1404      0.246     -0.570      0.569      -0.623       0.342
sepal width      3.2142      0.642      5.010      0.000       1.957       4.472
intercept       -8.0251      2.387     -3.362      0.001     -12.703      -3.347
================================================================================
"""

In [6]:
# Package 2: sklearn
from sklearn.linear_model import LogisticRegression
dat2 = dat.copy()
lr = LogisticRegression(random_state=0, C=1e6) # lower C means higher penalty, use convention in SVM
lr.fit(dat2.loc[:,dat2.columns!='target'], dat2['target'])
print('intercept: ', lr.intercept_)
dict(zip(dat2.columns[dat2.columns!='target'], lr.coef_[0]))

intercept:  [-8.0244352]


{'sepal length': -0.1403859441267881, 'sepal width': 3.214023049142017}

In [7]:
# sklearn with l2 regularization, C=25
lr = LogisticRegression(random_state=0, C=25, penalty='l2') # lower C means higher penalty, use convention in SVM
lr.fit(dat2.loc[:,dat2.columns!='target'], dat2['target'])
print('intercept: ', lr.intercept_)
dict(zip(dat2.columns[dat2.columns!='target'], lr.coef_[0]))

intercept:  [-6.49918279]


{'sepal length': -0.230651632407577, 'sepal width': 2.87903841342581}

We can see the result is slightly different, probably because one requires smaller different between iterations to determine the convergence. Therefore, we believe everything is correct here. 

### Model 2: Build the logistic regression from scratch

In [8]:
# review the dataset
dat.describe()

,sepal length,sepal width,target
count,150.000000,150.000000,150.000000
mean,5.843333,3.054000,0.666667
std,0.828066,0.433594,0.472984
min,4.300000,2.000000,0.000000
25%,5.100000,2.800000,0.000000
50%,5.800000,3.000000,1.000000
75%,6.400000,3.300000,1.000000
max,7.900000,4.400000,1.000000


In [9]:
# build sigmoid functions
def sigmoid(x):
    return(1/(1+math.exp(-x))) # we should raise exception for different x data type

In [10]:
# build loss function 
def lr_loss(y=1, p=0.5):
    if (y in [0,1]) and (0<p<1):
        return(-y*math.log(p)-(1-y)*math.log(1-p))
    else:
        raise ValueError('input y or p is out of bound.')

In [11]:
# let's train the model
def lr_train(data = dat, fit_intercept = True, random_state = 0, alpha = [0.05, 1, 1], tol = 1e-5, target = 'target', varList = []):
    # assume there is no column called 'intercept'
    if fit_intercept:
        dat['intercept'] = 1
        varList.append('intercept')
    # initiate beta based on random_state:
    random.seed(random_state) 
    init_beta = [0] * 3
    new_beta = [random.random() for i in range(len(varList))]
    # add two columns: predicted prob 
    data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], init_beta)), axis = 1)
    data['loss'] = data.apply(lambda row: lr_loss(y=row['target'], p=row['pred']), axis=1)
    # loop through 
    # but we will use vectorization
    while max(abs(np.array(new_beta) - np.array(init_beta)))>tol:
        init_beta = new_beta
        loss = np.sum(data[varList].mul(data['target']-data['pred'], axis=0), axis=0)*(1/data.shape[0])
        new_beta = np.sum([np.prod([loss, alpha], axis=0), init_beta], axis=0)
        data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], new_beta)), axis=1)
        print(new_beta)
    return(new_beta)    

In [ ]:
print(lr_train(data=dat, varList = ['sepal length', 'sepal width'])) # , 'petal length', 'petal width'

There is something wrong with this chunk of code:
    1. it could be parallel computed
    2. the running time is really volatile -> could be something wrong with the code here (*)

the result <br>
[-0.14102365473816966, 3.2122877033154436, -8.01551302010229]
<br>
It just takes too long to compute. The learning rate should be selected really carefully. 

Slightly different result, could be the difference in stopping rules.

### Model 3: add regularization in model 2

In [12]:
def lr_train_l2reg(data = dat, fit_intercept = True, C = 5e1, random_state = 0, alpha = [0.05, 1, 1], tol = 1e-5, target = 'target', varList = []):
    # assume there is no column called 'intercept'
    if fit_intercept:
        dat['intercept'] = 1
        varList.append('intercept')
    # initiate beta based on random_state:
    random.seed(random_state) 
    init_beta = [0] * 3
    new_beta = [random.random() for i in range(len(varList))]
    # new_beta = [0.26,  2.779, -1.2988, 2.703, -7.320] 
    # add two columns: predicted prob 
    data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], init_beta)), axis = 1)
    data['loss'] = data.apply(lambda row: lr_loss(y=row['target'], p=row['pred']), axis=1)
    # loop through
    while max(abs(np.array(new_beta) - np.array(init_beta)))>tol:
        init_beta = new_beta
        loss = np.subtract(np.sum(data[varList].mul(data['target']-data['pred'], axis=0), axis=0), np.dot(2/C,init_beta)) *(1/data.shape[0])
        new_beta = np.sum([np.prod([loss, alpha], axis=0), init_beta], axis=0)
        data['pred'] = data.apply(lambda row: sigmoid(np.dot(row[varList], new_beta)), axis=1)
        print(new_beta)
    return(new_beta)


In [ ]:
print(lr_train_l2reg(data=dat, varList = ['sepal length', 'sepal width'])) # , 'petal length', 'petal width'     

the result <br>
[-0.23119375274983178, 2.877706303593354, -6.492015452858157]
<br>
It just takes too long to compute. The learning rate should be selected really carefully. 